# Capitulo 7: Uso de Ejemplos / Prompting con Ejemplos (Few-Shot)

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Corregir la ruta de importacion para las pistas
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Leccion

**Darle a Claude ejemplos de como quieres que se comporte (o como no quieres que se comporte) es extremadamente efectivo** para:
- Obtener la respuesta correcta
- Obtener la respuesta en el formato correcto

Este tipo de prompting tambien se llama "**prompting con ejemplos (few-shot)**". Tambien podrias encontrar la frase "zero-shot" o "n-shot" o "one-shot". El numero de "shots" se refiere a cuantos ejemplos se usan dentro del prompt.

### Ejemplos

Imagina que eres un desarrollador intentando construir un "bot para padres" que responde preguntas de ninos. **La respuesta predeterminada de Claude es bastante formal y robotica**. Esto va a romper el corazon de un nino.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

Podrias tomarte el tiempo de describir el tono deseado, pero es mucho mas facil simplemente **darle a Claude algunos ejemplos de respuestas ideales**.

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

En el siguiente ejemplo de formato, podriamos guiar a Claude paso a paso a traves de un conjunto de instrucciones de formato sobre como extraer nombres y profesiones y luego formatearlos exactamente como queremos, o simplemente podemos **proporcionarle a Claude algunos ejemplos correctamente formateados y Claude puede extrapolar a partir de ahi**. Nota el `<individuals>` en el turno del `assistant` para encaminar a Claude por el camino correcto.

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Pre-llenado para la respuesta de Claude
PREFILL = "<individuals>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO:")
print(PROMPT)
print("\nTURNO DEL ASISTENTE:")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 7.1 - Formato de Correos via Ejemplos](#ejercicio-71---formato-de-correos-via-ejemplos)

### Ejercicio 7.1 - Formato de Correos via Ejemplos
Vamos a rehacer el Ejercicio 6.2, pero esta vez, vamos a editar el `PROMPT` para usar ejemplos "few-shot" de correos + clasificacion correcta (y formato) para que Claude genere la respuesta correcta. Queremos que la *ultima* letra del output de Claude sea la letra de la categoria.

Consulta los comentarios junto a cada correo en la lista `EMAILS` si olvidas que letra de categoria es correcta para cada correo.

Recuerda que estas son las categorias para los correos:
- (A) Pregunta de pre-venta
- (B) Articulo roto o defectuoso
- (C) Pregunta de facturacion
- (D) Otro (por favor explique)

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = """Please classify this email as either green or blue: {email}"""

# Pre-llenado para la respuesta de Claude
PREFILL = ""

# Contenido variable almacenado como una lista
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Articulo roto o defectuoso
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pregunta de pre-venta O (D) Otro (por favor explique)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Pregunta de facturacion
    "How did I get here I am not good with computer.  Halp." # (D) Otro (por favor explique)
]

# Categorizaciones correctas almacenadas como una lista de listas para acomodar la posibilidad de multiples categorizaciones correctas por correo
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterar a traves de la lista de correos
for i,email in enumerate(EMAILS):
    
    # Sustituir el texto del correo en la variable de marcador de posicion del correo
    formatted_prompt = PROMPT.format(email=email)
   
    # Obtener la respuesta de Claude
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Calificar la respuesta de Claude
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Imprimir la respuesta de Claude
    print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
    print("TURNO DEL USUARIO")
    print(formatted_prompt)
    print("\nTURNO DEL ASISTENTE")
    print(PREFILL)
    print("\n------------------------------------- Respuesta de Claude -------------------------------------")
    print(response)
    print("\n------------------------------------------ CALIFICACION ------------------------------------------")
    print("Este ejercicio se ha resuelto correctamente:", grade, "\n\n\n\n\n\n")

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Aun atascado? Ejecuta la celda de abajo para ver una solucion de ejemplo.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### Felicidades!

Si has resuelto todos los ejercicios hasta este punto, estas listo para pasar al siguiente capitulo. Feliz prompting!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompt mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Pre-llenado para la respuesta de Claude
PREFILL = "<individuals>"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print("TURNO DEL USUARIO:")
print(PROMPT)
print("\nTURNO DEL ASISTENTE:")
print(PREFILL)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))